In [1]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split

In [2]:

# Read config file
config = dict()
with open('./config/training_config.json') as f:
    config = json.load(f)

# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyperparameters
num_epochs = config['num_epochs']
batch_size = config['batch_size']
learning_rate = config['learning_rate']

In [3]:
# Load data
df = pd.read_csv(config['data_path'])
df = df.drop(config['drop_columns'], axis=1)
df = df.dropna()
df = df.reset_index(drop=True)
df = df.sort_values('time')
df = df.reset_index(drop=True)


In [4]:
df.head()

,time,open,high,low,close,volume,ema_22,ema_50,ema_200,rsi,kurtosis,mad,tos_stdevall,signal1
0,2020-02-03 04:00:00,9396.68,9432.61,9353.49,9372.99,14202.143,9356.411349,9189.706664,8416.197050,53.007172,0.032493,279.954432,336.223099,0.0
1,2020-02-03 08:00:00,9373.00,9407.00,9304.00,9356.72,22412.888,9356.438188,9196.256206,8425.555487,51.828757,0.287508,260.658208,316.831228,0.0
2,2020-02-03 12:00:00,9356.67,9378.82,9280.00,9283.16,23634.751,9350.066172,9199.664198,8434.088866,46.766572,0.493114,239.319800,295.114092,0.0
3,2020-02-03 16:00:00,9283.93,9348.00,9250.00,9331.11,26661.879,9348.417809,9204.818936,8443.014449,50.182294,0.859254,219.831376,276.379313,0.0
4,2020-02-03 20:00:00,9332.16,9360.00,9263.00,9300.64,14104.385,9344.263217,9208.576624,8451.548037,48.071461,1.019641,198.044056,253.643963,0.0


In [5]:
# Group data by windows_size
windows_size = config['windows_size']
group = []
for t in range(len(df) - windows_size+1):group.append(df.iloc[t:t+windows_size])

In [6]:
group = np.array(group)
group.shape

(6797, 5, 14)

In [7]:
#Balance data group[:,:,-1][i][-1] > 0 and group[:,:,-1][i][-1] == 0 and group[:,:,-1][i][-1] < 0
group_hold = []
group_buy = []
group_sell = []

for i in range(len(group)):
    if group[:,:,-1][i][-1] > 0:
        group_buy.append(group[i])
    elif group[:,:,-1][i][-1] == 0:
        group_hold.append(group[i])
    elif group[:,:,-1][i][-1] < 0:
        group_sell.append(group[i])

group_hold = np.array(group_hold)
group_buy = np.array(group_buy)
group_sell = np.array(group_sell)


In [8]:
print(group_hold.shape)
print(group_buy.shape)
print(group_sell.shape)

(6729, 5, 14)
(23, 5, 14)
(45, 5, 14)


In [9]:
group_buy[1]

array([['2020-07-06 04:00:00', 9087.31, 9238.23, 9080.14, 9199.71,
        44067.463, 9098.58434232904, 9133.233560836425,
        9249.687302320415, 62.2615062350233, -0.0808134972520387,
        50.46530400000003, 62.92841235264658, 0.0],
       ['2020-07-06 08:00:00', 9199.72, 9230.0, 9171.0, 9221.24,
        18339.389, 9109.250051691732, 9136.6847937448, 9249.404244585883,
        64.22037633731235, -0.4519611864033031, 49.84814399999999,
        61.02846656023757, 0.0],
       ['2020-07-06 12:00:00', 9221.24, 9320.0, 9200.0, 9302.99,
        54834.379, 9126.09700371854, 9143.206566539122,
        9249.937436182046, 70.48496776769062, 0.0069577788655023,
        51.69635199999997, 64.54351985051602, 0.0],
       ['2020-07-06 16:00:00', 9302.99, 9348.0, 9261.0, 9287.38,
        39263.344, 9140.12161209084, 9148.860426674844,
        9250.309999006104, 68.03538751839157, -0.0547802397337994,
        53.48491999999984, 67.4028979653135, 0.0],
       ['2020-07-06 20:00:00', 9287.83, 93